In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from LxmlSoup import LxmlSoup
import nltk
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering
from sklearn.decomposition import PCA
from nltk.stem.snowball import SnowballStemmer
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import pyprind
import string
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Имя пользователя или название беседы
NAME = None

# Используемая модель (DBSCAN или KMeans)
# При некорректном имени по умолчанию будет использован KMeans
MODEL_TYPE = 'Spectral'

# Максимальное количество кластеров (для KMeans и Spectral)
MAX_CLUSTERS = 150

# eta для DBSCAN
ETA = 0.5

# Использовать PCA перед кластеризацией (True или False)
USE_PCA =False

# Минимальный порог для TF-IDF
MIN_DF = 5

# Количество точек для визуализации PCA
POINT_COUNT = 1000

# Сохранять информацию в csv файле после парсинга (True или False)
# Если False, при каждом новом запуске программы будет осуществляться повторный парсинг, что может занимать много времени
SAVE_TO_CSV = True

# Чтение информации из csv файла (True или False)
# Если False, существующие csv файлы будут проигнорированы и перезаписаны (если SAVE_TO_CSV = True). Используйте если обновили данные архива
READ_FROM_CSV = True

# Количество компонент для PCA (график будет выведен только для 2 или 3 компонент)
PCA_COMPONENTS = 3

# Использовать ли эмбеддинги BERT вместо TF-IDF векторизации (True или False)
USE_BERT = True

# Директория с моделями hugging face (если None, то будет использована директория по умолчанию)
HF_FOLDER = None

In [ ]:
def find_folder(name, parent_directory):
    folder_path = os.path.join(parent_directory, 'messages')

    directory_num = 0
    
    for root, dirs, files in os.walk(folder_path):
        for directory in dirs:
            dir_path = os.path.join(root, directory)
            file_path = os.path.join(dir_path, "messages0.html")
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    file_contents = file.read()
                    soup = BeautifulSoup(file_contents, 'html.parser')
                    divs = soup.find_all('div', class_='ui_crumb')
                    title = divs[0].text
                    if name in title:
                        directory_num = directory
                        break
    return directory_num


In [ ]:
def get_messages(messages, messages_directory):
    month_dict = {
        'янв': 1,
        'фев': 2,
        'мар': 3,
        'апр': 4,
        'мая': 5,
        'июн': 6,
        'июл': 7,
        'авг': 8,
        'сен': 9,
        'окт': 10,
        'ноя': 11,
        'дек': 12}
    count = 0
    for _, _, files in os.walk(messages_directory):
        count += len(files)
    pbar = pyprind.ProgBar(count)
    for filename in os.listdir(messages_directory):
        if os.path.isfile(os.path.join(messages_directory, filename)):
            with open(os.path.join(messages_directory, filename), 'r') as file:
                content = file.read()
                #soup = BeautifulSoup(content, 'html.parser')
                soup = LxmlSoup(content)
                divs = soup.find_all('div', class_='message')
                for div in divs:
                    div_content = div.text()
                    div_content = div_content.split('\n')
                    div_dict = {}
                    
                    div_dict['text'] = div_content[1]
                    author_and_time = div_content[0].split(',')
                    
                    div_dict['author'] = author_and_time[0]
                    datetime = author_and_time[1].split(' в ')
                    div_dict['time'] = datetime[1]
                    date = datetime[0].split(' ')
                    div_dict['day'] = date[1]
                    div_dict['month'] = month_dict[date[2]]
                    div_dict['year'] = date[3]
                    messages.loc[len(messages)] = div_dict
                pbar.update()
    return messages

In [ ]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

messages_folder = find_folder(NAME, parent_directory)
if messages_folder == 0:
    raise ValueError("Такой чат не найден")
path_to_messages = 'messages/'+str(messages_folder)
print('Чат найден, id: ', messages_folder)
messages_directory = os.path.join(parent_directory, path_to_messages)

print(messages_directory)
filename = current_directory + '/savedata/' + str(messages_folder) + '.csv'
if os.path.exists(filename) and READ_FROM_CSV:
    messages = pd.read_csv(filename)
    print('Информация о чате извлечена из сохраненного файла')
else:
    print('Информация о чате не найдена в сохраненных файла. Начат парсинг html файлов для извлечения информации')
    messages = pd.DataFrame(columns=['day', 'month', 'year', 'time', 'author', 'text'])
    start_time = time()
    messages = get_messages(messages, messages_directory)
    end_time = time()
    print('Время парсинга: ', end_time - start_time, ' секунд')
    if SAVE_TO_CSV:
        os.chdir(current_directory)
        csv_path = 'savedata/' + str(messages_folder) + '.csv'
        messages.to_csv(csv_path, encoding='utf-8-sig', index=False)
        print('Информация о чате извлечена и сохранена в csv файл')
    else:
        print('Информация о чате извлечена. У вас установлен флаг SAVE_TO_CSV = False, поэтому информация о чате не будет сохранена в csv файл. При следующем запуске будет осуществлен повторный парсинг, что может занять продолжительное время')

messages.head()

In [ ]:
messages.shape

In [ ]:
messages['date'] = pd.to_datetime(messages['month'].astype(str) + '-' + messages['year'].astype(str))
max_year = messages['year'].max()
messages_in_year = messages[messages['year'] == max_year]
max_month = messages_in_year['month'].max()

count_by_date = messages[(messages['year'] != max_year) | (messages['month'] != max_month)].groupby('date').size()
count_by_date.plot(kind='line')

plt.xlabel('Даты')
plt.ylabel('Количество сообщений')
plt.title('Количество сообщений в месяц')
plt.show()

In [ ]:
authors = messages['author'].unique()
for author in authors:
    count_by_date_authors = messages[((messages['year'] != max_year) | (messages['month'] != max_month)) & (messages['author']==author)].groupby('date').size()
    count_by_date_authors.plot(kind='line', label=author)


plt.xlabel('Даты')
plt.ylabel('Количество сообщений')
plt.legend()
plt.title('Количество сообщений в месяц (по отправителям)')
plt.show()

In [ ]:
avg_len = messages.groupby('author')['text'].apply(lambda x: x.str.len().mean())
print(avg_len)

In [ ]:
df_cluster = messages['text']
df_cluster.head()

In [ ]:
df_cluster = df_cluster.fillna('None')

In [ ]:
def tokenize_row_stopwords(row):
    stemmer = SnowballStemmer("russian")
    nltk_stopwords = stopwords.words('russian')
    tokenizer = nltk.tokenize.word_tokenize(row, language='russian')
    tokens = [i for i in tokenizer if (i not in string.punctuation) and (i not in nltk_stopwords)]
    return [stemmer.stem(word) for word in tokens]

def text_vectorization_stopwords(df, n_grams = (1, 2)):
    vectorizer = TfidfVectorizer(min_df=MIN_DF, max_df=0.5, ngram_range=n_grams, tokenizer=lambda text: tokenize_row_stopwords(text))
    features = vectorizer.fit_transform(df)
    return pd.DataFrame(features.todense(), columns=vectorizer.get_feature_names_out()), vectorizer

In [ ]:
class BertVectorizer:
    def __init__(self, dataset):
        self.dataset = dataset
        if HF_FOLDER:
            self.model = SentenceTransformer('LaBSE', cache_folder=HF_FOLDER)
        else:
            self.model = SentenceTransformer('LaBSE')
            
    def vectorize(self):
        embeddings =  self.model.encode(self.dataset, show_progress_bar=True)
        return pd.DataFrame(embeddings)
        

In [ ]:
if USE_BERT == False:
    df_cluster_vector, vectorizer = text_vectorization_stopwords(df_cluster, n_grams=(1, 2))
else:
    bert = BertVectorizer(df_cluster)
    df_cluster_vector = bert.vectorize()

In [ ]:
pca = PCA(n_components=PCA_COMPONENTS)
pca.fit(df_cluster_vector)
if USE_PCA:
    df_cluster_vector = pca.transform(df_cluster_vector)

In [ ]:
if MODEL_TYPE == 'DBSCAN':
    model = DBSCAN(eps=ETA, min_samples=5)
elif MODEL_TYPE == 'KMeans':
    model = KMeans(n_clusters=MAX_CLUSTERS, random_state=0)
elif MODEL_TYPE == 'Spectral':
    model = SpectralClustering(n_clusters=MAX_CLUSTERS, random_state=0, affinity='nearest_neighbors')
clusters = model.fit_predict(df_cluster_vector)


In [ ]:
df_with_clusters = pd.concat([df_cluster, pd.Series(clusters, name='cluster_label')], axis=1)
df_with_clusters.shape

In [ ]:
unique_labels = df_with_clusters['cluster_label'].unique()
for label in unique_labels:
    cluster_texts = df_with_clusters[df_with_clusters['cluster_label'] == label]['text']
    print(f"Кластер {label}:")
    print(cluster_texts.head(20))
    print("Количество сообщений в кластере:", len(cluster_texts))
    print("---------------------------------------------")

In [ ]:
def plot_pca_reduced(X, y, PCA_COMPONENTS):
    unique_labels = y.unique()
    colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
    fig = plt.figure(figsize=(8, 6))
    if PCA_COMPONENTS == 3:
        ax = fig.add_subplot(111, projection='3d')
    for cl, color in zip(unique_labels, colors):
        indices = np.where(y == cl)
        points = X[indices]
        if PCA_COMPONENTS == 3:
            ax.scatter(points[:, 0], points[:, 1], points[:, 2], color=color, label=cl)
        else:
            plt.scatter(points[:, 0], points[:, 1], color=color, label=cl)
    if PCA_COMPONENTS == 3:
        ax.set_xlabel('PCA Component 1')
        ax.set_ylabel('PCA Component 2')
        ax.set_zlabel('PCA Component 3')
        ax.set_title('PCA Reduced Representation of X')
    else:
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.title('PCA Reduced Representation of X')
    plt.show()
    
def plot_pca(X, y, pca, PCA_COMPONENTS):
    X_reduced = pca.transform(X)
    plot_pca_reduced(X_reduced, y, PCA_COMPONENTS)
    

In [ ]:
if PCA_COMPONENTS == 2 or PCA_COMPONENTS == 3:
    if USE_PCA:
        plot_pca_reduced(df_cluster_vector[:POINT_COUNT], df_with_clusters['cluster_label'][:POINT_COUNT], PCA_COMPONENTS)
    else:
        plot_pca(df_cluster_vector[:POINT_COUNT], df_with_clusters['cluster_label'][:POINT_COUNT], pca, PCA_COMPONENTS)